In [2]:
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

In [3]:
# 문서 준비 및 분할

In [8]:
loader = PyPDFLoader('../../assets/pdf/2024_KB_부동산_보고서_최종.pdf')
documents= loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print('분할된 청크의 수 : ', len(chunks))

분할된 청크의 수 :  135


In [9]:
# 임베딩 생성 및 Chroma 저장

In [11]:
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory='../../assets/embedding/ch02/')
print('문서의 수 : ', chroma_db._collection.count())

문서의 수 :  135


In [12]:
# 리트리버 생성

In [13]:
retriever = chroma_db.as_retriever(search_kwargs={'k':3})

In [14]:
# 프롬프트 템플릿 설정

In [17]:
template = """
당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트 : {context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', template),
        ('placeholder', '{chat_history}'),
        ('human', '{question}')
    ]
)
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

In [18]:
print(prompt.format(context='컨텍스트 예시',
                    chat_history=['대화 기록 예시 1', '대화 기록 예시 2'],
                    question='질문 예시'))

System: 
당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트 : 컨텍스트 예시

Human: 대화 기록 예시 1
Human: 대화 기록 예시 2
Human: 질문 예시


In [21]:
# 참고 파일 하나로

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
# 체인 구성

In [22]:
chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x['question']))
    )
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
# 대화 기록 및 메모리 설정

In [30]:
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [31]:
# 챗봇 실행 함수 정의

In [32]:
def chat_with_bot():
    session_id='user_session'
    print('KB 부동산 보고서 챗봇입니다. 질문해주세요! (종료하시려면 "quit" 입력)')
    while True:
        user_input = input('사용자 : ')
        if user_input.lower() == 'quit':
            break
        response = chain_with_memory.invoke(
            {'question':user_input},
            {'configurable':{'session_id':session_id}}
        )

        print('챗봇 : ', response)

In [33]:
chat_with_bot()

KB 부동산 보고서 챗봇입니다. 질문해주세요! (종료하시려면 "quit" 입력)


사용자 :  랭체인 챗봇 만들기에 대해 실습중입니다.


챗봇 :  랭체인 챗봇을 만들기 위한 실습을 진행 중이시군요! 랭체인(LangChain)은 자연어 처리(NLP)와 관련된 다양한 작업을 수행할 수 있는 강력한 프레임워크입니다. 챗봇을 만들기 위해 다음과 같은 단계를 고려해 보세요:

1. **환경 설정**:
   - Python과 필요한 라이브러리(예: `langchain`, `openai`, `flask` 등)를 설치합니다.
   - API 키(예: OpenAI API 키)를 준비합니다.

2. **기본 구조 설계**:
   - 챗봇의 기본 구조를 설계합니다. 사용자 입력을 받고, 이를 처리하여 응답을 생성하는 흐름을 정의합니다.

3. **LangChain 사용**:
   - LangChain의 기능을 활용하여 자연어 처리 모델을 설정합니다. 예를 들어, OpenAI의 GPT 모델을 사용할 수 있습니다.
   - LangChain의 `LLMChain`을 사용하여 입력을 처리하고 응답을 생성하는 로직을 구현합니다.

4. **대화 관리**:
   - 대화의 맥락을 유지하기 위해 대화 상태를 관리하는 방법을 고려합니다. 예를 들어, 이전 대화 내용을 저장하고 이를 기반으로 응답을 생성할 수 있습니다.

5. **테스트 및 개선**:
   - 챗봇을 테스트하여 사용자와의 상호작용을 확인합니다. 피드백을 바탕으로 개선할 부분을 찾아 수정합니다.

6. **배포**:
   - 챗봇을 웹 애플리케이션으로 배포하거나, 메신저 플랫폼(예: Slack, Discord 등)에 통합할 수 있습니다.

이러한 단계를 통해 랭체인 챗봇을 성공적으로 만들 수 있을 것입니다. 구체적인 코드나 예제가 필요하시면 말씀해 주세요!


사용자 :  방금 제가 뭐라고 했나요?


챗봇 :  당신은 "랭체인 챗봇 만들기에 대해 실습중입니다."라고 말씀하셨습니다. 랭체인 챗봇을 만들기 위한 실습을 진행하고 계신 것에 대해 언급하셨습니다. 추가로 궁금한 점이나 도움이 필요하신 부분이 있으시면 말씀해 주세요!


사용자 :  수도권 주택 매매 전망


챗봇 :  2024 KB 부동산 보고서에 따르면, 수도권 주택시장은 전반적으로 침체 상태에 있으며, 특히 강남권과 같은 선호 지역에서는 상대적으로 강세를 보이고 있습니다. 다음은 수도권 주택 매매 전망에 대한 주요 내용입니다:

1. **거래 회복의 어려움**: 높은 기준금리와 주택 매매가격, DSR(총부채상환비율) 규제 등으로 인해 매수자들의 구매 여력이 회복되지 않고 있습니다. 매도자와 매수자 간의 희망가격 차이가 여전히 존재하여 거래가 원활하게 이루어지지 않고 있습니다.

2. **정부 규제 완화**: 2023년 이후 정부의 다양한 규제 완화가 이루어졌지만, 이는 매도자들의 기대 심리를 높이는 데 그치고 있습니다. 재건축 규제 완화와 같은 호재가 있지만, 전반적인 주택 경기 위축 요소도 여전히 존재합니다.

3. **지역별 차별화**: 수도권 내에서도 지역별로 매매 시장의 차별화가 나타나고 있습니다. 예를 들어, 강남구는 정부 규제 완화와 주택 수요 집중으로 긍정적인 기대감을 유지하고 있는 반면, 양천구는 매매시장 위축이 지속되고 있습니다.

4. **전세 시장과의 연관성**: 전세가격 하락이 매매시장에 미치는 영향도 고려해야 합니다. 대규모 단지의 입주가 예정되어 있는 지역에서는 전세가격 하락이 매매에 영향을 줄 수 있습니다.

5. **전반적인 전망**: 하반기 이후 거래 침체 현상이 다소 완화될 가능성이 있지만, 전반적인 회복세는 제한적일 것으로 보입니다. 매도자와 매수자 간의 가격 격차 축소와 매수세 회복이 중요합니다.

이러한 요소들을 종합적으로 고려할 때, 수도권 주택 매매 시장은 여전히 불확실성이 크고, 지역별로 상이한 상황이 전개될 것으로 예상됩니다. 추가적인 질문이 있으시면 말씀해 주세요!


사용자 :  노후계획도시가 어디인지 궁금해


챗봇 :  노후계획도시는 주로 1기 신도시를 포함한 지역들로, 1980년대 말에 대규모로 건설된 주택 단지들입니다. 2024 KB 부동산 보고서에 따르면, 노후계획도시 특별법의 적용을 받는 주요 지역은 다음과 같습니다:

1. **분당** (경기도 성남시)
2. **일산** (경기도 고양시)
3. **평촌** (경기도 안양시)
4. **산본** (경기도 군포시)
5. **중동** (경기도 부천시)

이들 지역은 조성된 지 20년 이상 경과하여 재정비가 필요한 상황이며, 노후계획도시 특별법에 따라 정비사업 규제가 완화될 예정입니다. 이 법안은 2024년 4월 27일부터 시행될 예정이며, 해당 지역의 재건축 및 정비사업에 긍정적인 영향을 미칠 것으로 기대되고 있습니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!


사용자 :  quit
